In [106]:
import numpy as np
import pandas as pd
import warnings

warnings.filterwarnings("ignore")

## Для начала я немного поработаю с файлом, в котором хранятся данные
### Первым делом узнаем кодировку файла

In [61]:
# Импортируем детектор распознавания кодировки
from chardet.universaldetector import UniversalDetector

detector = UniversalDetector()
with open('pluginfile.txt', 'rb') as fh:
    for line in fh:
        detector.feed(line)
        if detector.done:
            break
    detector.close()
    
print(detector.result)

{'encoding': 'MacCyrillic', 'confidence': 0.9451935059219918, 'language': 'Russian'}


### В файле столбцы разделяются с помощью табуляции, а знак, отделяющий целое число от дробной части это запятая, укажем эти параметры при чтении файла

In [62]:
df = pd.read_csv('pluginfile.txt', sep='\t', decimal=',', encoding='MacCyrillic')
df

,ƒлина чашелистика,Ўирина чашелистика,ƒлина лепестка,Ўирина лепестка,ласс
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa
...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica


### Для удобства я переименую названия колонок

In [63]:
df.columns = [
    "Длина чашелистика",
    "Ширина чашелистика",
    "Длина лепестка",
    "Ширина лепестка",
    "Класс"
]

df.head()

,Длина чашелистика,Ширина чашелистика,Длина лепестка,Ширина лепестка,Класс
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


### Теперь заменим метки класса на числа

In [64]:
df['Класс'].unique()

array(['Iris-setosa', 'Iris-versicolor', 'Iris-virginica'], dtype=object)

In [65]:
class_number = {
    'Iris-setosa': 0,
    'Iris-versicolor': 1,
    'Iris-virginica': 2
}

df['Класс'] = df['Класс'].apply(lambda x: class_number[x])

df.head()

,Длина чашелистика,Ширина чашелистика,Длина лепестка,Ширина лепестка,Класс
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


#### P.S. так то лучше)

### Теперь импортируем KNN модель, функцию разделения выборки, GridSearchCV и разные метрики из библиотеки sckelarn

In [85]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### Подготавливаем данные для модели, перемешиваем и разделяем на обучающую и тестовую выборки
### В тестовой выборке будет 20% всех данных

In [96]:
X_train, X_test, y_train, y_test = train_test_split(df.drop('Класс', axis=1), df['Класс'], test_size=0.2)

print(f'Размер обучающей выборки: {X_train.shape}')
print(f'Размер тестовой выборки: {X_test.shape}')

X_train.head()

Размер обучающей выборки: (120, 4)
Размер тестовой выборки: (30, 4)


,Длина чашелистика,Ширина чашелистика,Длина лепестка,Ширина лепестка
102,7.1,3.0,5.9,2.1
8,4.4,2.9,1.4,0.2
96,5.7,2.9,4.2,1.3
38,4.4,3.0,1.3,0.2
55,5.7,2.8,4.5,1.3


### Создадим модель KNN и обучим ее с помощью GridSearchCV с указанными параметрами

In [107]:
# Создаем модель KNN
model = KNeighborsClassifier()

# Создаем список с перебираемыми параметрами для GridSearchCV
params = {
    'n_neighbors': range(3, 9),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_thee', 'kd_tree', 'brute'],
    'leaf_size': range(25, 42),
    'n_jobs': [-1]
}

# Обучаем модель на каждой комбинации параметров
clf = GridSearchCV(model, params)
clf.fit(X_train, y_train)

# Записываем модель с лучшими показателями в model
model = clf.best_estimator_

print(f'Лучший показатель: {clf.best_score_}')
print(f'Лучшие параметры: {clf.best_params_}')

Лучший показатель: 0.975
Лучшие параметры: {'algorithm': 'auto', 'leaf_size': 25, 'n_jobs': -1, 'n_neighbors': 8, 'weights': 'distance'}


In [108]:
predict = model.predict(X_test)

acc_score = accuracy_score(predict, y_test)

print(f'Показатель метрики accuracy: {acc_score}')

Показатель метрики accuracy: 0.9666666666666667


In [109]:
predict_df = pd.DataFrame(y_test).rename(columns={'Класс': 'true'})
predict_df['predict'] = predict

predict_df

,true,predict
47,0,0
142,2,2
94,1,1
23,0,0
60,1,1
24,0,0
85,1,1
90,1,1
6,0,0
129,2,2
